<a href="https://colab.research.google.com/github/DigiBP/Team-Chillon-Castle/blob/master/prescription_parser_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Input
Upload file with text.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving prescription_melissa.jpeg to prescription_melissa.jpeg


### Install Pre-requisites
- tesseract linux binary
- pytesseract

In [ ]:
!sudo apt install tesseract-ocr -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [ ]:
!pip install pytesseract

### OCR

In [ ]:
import pytesseract
from PIL import Image
file_content = pytesseract.image_to_string(Image.open('prescription_melissa.jpeg'))

### Post-processing to Google Sheet

In [ ]:
# Run this cell only once
from collections import OrderedDict
final_prescriptions = OrderedDict()

In [ ]:
def parse_prescription(file_content):
  file_content_str = [elem.strip() for elem in file_content.split('\n')]
  prescription_row = {}
  for elem in map(str,file_content_str):
    if ':' in elem:
      key, val = elem.split(':')
      prescription_row[key] = val
  return prescription_row



In [ ]:
# file_content = open('out.txt.txt', 'rb').readlines()
prescription_row=parse_prescription(file_content)

In [ ]:
final_prescriptions[len(final_prescriptions)] = prescription_row

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame.from_dict(final_prescriptions, orient="index")
df

,Name,Zip,ePharmacy,Mail,Drug,Medication Plan,Package Size
0,Melissa,4058,no,melissaforgetful@gmail.com,Tirosint,1-0-0,90


In [ ]:
#Export to Google Sheets / Part 1 Auth
!pip install --upgrade --quiet gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
#Export to Google Sheets / Part 2 Export
from gspread_dataframe import get_as_dataframe, set_with_dataframe

patient_data = gc.open_by_url('https://docs.google.com/spreadsheets/d/1tbLYRSAfDMbr8cueQInhWVA6ISAD2c8kc9X7EonvOX8/edit#gid=0')
ws1 = patient_data.get_worksheet(0)
ws1.update_cell(2,2,df.iloc[0]['Name'])
ws1.update_cell(2,3,df.iloc[0]['Zip'])
ws1.update_cell(2,4,df.iloc[0]['ePharmacy'])
ws1.update_cell(2,5,df.iloc[0]['Mail'])
ws1.update_cell(2,6,df.iloc[0]['Drug'])
ws1.update_cell(2,7,df.iloc[0]['Medication Plan'])
ws1.update_cell(2,8,df.iloc[0]['Package Size'])

{'spreadsheetId': '1tbLYRSAfDMbr8cueQInhWVA6ISAD2c8kc9X7EonvOX8',
 'updatedCells': 1,
 'updatedColumns': 1,
 'updatedRange': 'Tabellenblatt1!H2',
 'updatedRows': 1}